In [ ]:
from pathlib import Path

import Metashape

from result import Ok, Err, Result

from benthoscan.registration import PointCloud, PointCloudLoader
from benthoscan.backends import metashape as backend
from benthoscan.utils.log import logger

In [ ]:
PATHS: dict[str, Path] = {
    "DOCUMENT_IN": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
    ),
    "DOCUMENT_OUT": Path(
        "/data/kingston_snv_01/acfr_revisits_metashape_projects_test/r23685bc_working_version_saved.psz"
    ),
    "CACHE": Path("/home/martin/dev/benthoscan/.cache/"),
}

result: Result[str, str] = backend.load_project(PATHS.get("DOCUMENT_IN"))
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

backend.log_internal_data()

### Define disparity estimator

In [ ]:
import cv2
import numpy as np


def sgbm_default_parameters() -> dict:
    """
    Returns default parameters for semi-global block matching.
    Adopted from: https://github.com/eborboihuc/stereo-matching/blob/master/sgbm.py
    """

    min_disparity: int = 0
    num_disparities: int = 7 * 16  # NOTE: 112
    block_size: int = 5
    window_size: int = 3
    disp12_max_diff: int = 1
    uniqueness_ratio: int = 10
    speckle_window_size: int = 50
    speckle_range: int = 1
    pre_filter_cap: int = 63
    mode: object = cv2.STEREO_SGBM_MODE_SGBM_3WAY

    p1: int = 8 * 1 * block_size**2  # **2 # Smoothness
    p2: int = 32 * 1 * block_size**2  # **2 # Smoothness -

    return {
        "minDisparity": min_disparity,
        "numDisparities": num_disparities,
        "blockSize": block_size,
        "P1": p1,
        "P2": p2,
        "disp12MaxDiff": disp12_max_diff,
        "uniquenessRatio": uniqueness_ratio,
        "speckleWindowSize": speckle_window_size,
        "speckleRange": speckle_range,
        "preFilterCap": pre_filter_cap,
        "mode": mode,
    }


def filter_disparity(
    estimator: object,
    image_left: np.ndarray,
    disparity_left: np.ndarray,
    disparity_right: np.ndarray,
) -> np.ndarray:
    """Filter disparity map based on WLS. Returns the disparity map as 16-bit pixels."""

    filtered_disparity_left = estimator.filter(
        disparity_left,
        image_left,
        None,
        disparity_right,
    )

    return filtered_disparity_left


def compute_disparity_sgbm(
    image_left: np.ndarray, image_right: np.ndarray, smooth: bool = False
):
    """Computes the disparity for an image pair using OpenCVs SGBM algorithm."""

    parameters: dict = sgbm_default_parameters()

    left_matcher = cv2.StereoSGBM_create(**parameters)
    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

    # Disparity is represented with 16-bit integers. To convert to pixel value cast to float
    # and divide by 16, i.e.:  disparity.astype(np.float32) / 16.0
    disparity_left: np.ndarray = left_matcher.compute(image_left, image_right)
    disparity_right: np.ndarray = right_matcher.compute(image_right, image_left)

    disparity_left: np.ndarray = disparity_left.astype(np.float32) / 16.0
    disparity_right: np.ndarray = disparity_right.astype(np.float32) / 16.0

    if smooth:
        # FILTER Parameters
        lmbda: int = 100  # regularization parameter 500
        sigma: float = 0.2

        disparity_filter = cv2.ximgproc.createDisparityWLSFilter(
            matcher_left=left_matcher
        )
        disparity_filter.setLambda(lmbda)
        disparity_filter.setSigmaColor(sigma)

        disparity_left: np.ndarray = filter_disparity(
            disparity_filter, image_left, disparity_left, disparity_right
        )
        disparity_right: np.ndarray = filter_disparity(
            disparity_filter, image_right, disparity_right, disparity_left
        )

    assert disparity_left.dtype == np.float32, "invalid disparity type: expected int16"
    assert disparity_right.dtype == np.float32, "invalid disparity type: expected int16"

    return disparity_left, disparity_right

### Define overall stereo depth estimation process

In [ ]:
import cv2

import plotly.express as px
import plotly.graph_objects as go

from benthoscan.backends.metashape.camera_helpers import (
    SensorPair,
    CameraPair,
    StereoGroup,
    compute_camera_calibration,
    compute_stereo_calibration,
    image_to_array,
    get_stereo_groups,
)

from benthoscan.geometry.stereo import (
    CameraCalibration,
    StereoCalibration,
    RectifyingHomography,
    RectifyingPixelMap,
    compute_rectifying_homographies,
    compute_rectifying_pixel_maps,
    rectify_image_pair,
)


def disparity_to_range(
    disparity_map: np.ndarray, focal_length: float, baseline: float
) -> np.ndarray:
    """Converts disparity to range."""
    INVALID_RANGE: float = 0.0

    inverse_disparity: np.ndarray = 1.0 / disparity_map

    range_map: np.ndarray = inverse_disparity * focal_length * baseline
    range_map: np.ndarray = np.where(range_map < 0.0, INVALID_RANGE, range_map)

    return range_map


def process_stereo_images(
    chunk: Metashape.Chunk, sensors: SensorPair, camera_pairs: list[CameraPair]
):
    """Get the stereo images"""

    # Convert a pair of Metashape sensors to a stereo calibration
    calibration: StereoCalibration = compute_stereo_calibration(sensors)

    focal_length: float = calibration.master.focal_length
    baseline: float = calibration.baseline

    logger.info(focal_length)
    logger.info(baseline)

    # Estimate homographies and pixel maps
    homographies: RectifyingHomography = compute_rectifying_homographies(calibration)
    pixel_maps: RectifyingPixelMap = compute_rectifying_pixel_maps(
        calibration, homographies
    )

    for index, camera_pair in enumerate(camera_pairs):

        logger.info(f"Camera: {camera_pair.master.label}")

        master_image: np.ndarray = np.squeeze(
            image_to_array(camera_pair.master.image())
        )
        slave_image: np.ndarray = np.squeeze(image_to_array(camera_pair.slave.image()))

        # Convert RGB to grayscale
        master_image: np.ndarray = cv2.cvtColor(master_image, cv2.COLOR_RGB2GRAY)

        rectified_master, rectified_slave = rectify_image_pair(
            master_image,
            slave_image,
            pixel_maps,
        )

        disparity_maps: tuple[np.ndarray, np.ndarray] = compute_disparity_sgbm(
            rectified_master, rectified_slave, smooth=False
        )
        smooth_disparity_maps: tuple[np.ndarray, np.ndarray] = compute_disparity_sgbm(
            rectified_master, rectified_slave, smooth=True
        )

        # Convert disparity to range estimates
        range_map: np.ndarray = disparity_to_range(disparity_maps[0], 1720.0, baseline)
        smooth_range_map: np.ndarray = disparity_to_range(
            smooth_disparity_maps[0], 1720.0, baseline
        )

        figures: dict = {
            "image": px.imshow(
                master_image, origin="upper", color_continuous_scale="gray"
            ),
            "range": px.imshow(range_map, origin="upper", zmin=2.0, zmax=2.7),
            "smooth_range": px.imshow(
                smooth_range_map, origin="upper", zmin=2.0, zmax=2.7
            ),
        }

        for key, figure in figures.items():
            figure.show()

        raise NotImplementedError("process_stereo_images is not implemented")


def export_stereo_range_maps(chunk: Metashape.Chunk, directory: Path) -> None:
    """Export range maps based on a stereo camera setup."""

    # Get pairs of sensors and cameras (master-slaves)
    stereo_groups: list[StereoGroup] = get_stereo_groups(chunk)

    for group in stereo_groups:
        process_stereo_images(chunk, group.sensor_pair, group.camera_pairs)

    raise NotImplementedError

### Test stereo depth estimation on camera pairs from Metashape chunks

In [ ]:
document: Metashape.Document = backend.context._backend_data.get("document")

target_labels: list[str] = ["r23685bc_20100605_021022"]
target_chunks: list[Metashape.Chunk] = [
    chunk for chunk in document.chunks if chunk.label in target_labels
]

output_root: Path = Path("/data/kingston_snv_01/stereo_range_maps")

# Generate range maps based on stereo pairs
for chunk in target_chunks:
    output_directory: Path = output_root / Path(f"{chunk.label}_range_maps")
    export_stereo_range_maps(chunk, output_directory)